<a href="https://colab.research.google.com/github/deanhadzi/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/module1-statistics-probability-and-inference/LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### YOUR CODE STARTS HERE
# Importing necessary libraries to complete the homework.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel, ttest_1samp

In [0]:
# Making a list to change the column names in the dataframe. 

column_names = ['Party', 'Vote 1', 'Vote 2', 'Vote 3',
                'Vote 4', 'Vote 5', 'Vote 6', 'Vote 7', 
                'Vote 8', 'Vote 9', 'Vote 10', 'Vote 11',
                'Vote 12', 'Vote 13', 'Vote 14', 'Vote 15', 
                'Vote 16']

In [3]:
# Importing dataframe, changing column names
# I changed ? votes by NaNs, Yes votes became 1, No votes became 0.
# Cheking the shape and format of the dataframe. 

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data', header=None, names=column_names, na_values='?')
df.replace({'y': 1, 'n': 0}, inplace=True)
print(df.shape)
df.head()

(435, 17)


,Party,Vote 1,Vote 2,Vote 3,Vote 4,Vote 5,Vote 6,Vote 7,Vote 8,Vote 9,Vote 10,Vote 11,Vote 12,Vote 13,Vote 14,Vote 15,Vote 16
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [4]:
# In order to calculate the two sample T test we are spliting the dataframe into 2 samples.
# One sample is Republican representatives.
# Dropping Party column from the subset dataframe as we don't need it. 

rep = df[df['Party'] == 'republican']
rep = rep.iloc[:, 1:]
rep.head()

,Vote 1,Vote 2,Vote 3,Vote 4,Vote 5,Vote 6,Vote 7,Vote 8,Vote 9,Vote 10,Vote 11,Vote 12,Vote 13,Vote 14,Vote 15,Vote 16
0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [5]:
# Second sample is Democrat representatives. 

dem = df[df['Party'] == 'democrat']
dem = dem.iloc[:, 1:]
dem.head()

,Vote 1,Vote 2,Vote 3,Vote 4,Vote 5,Vote 6,Vote 7,Vote 8,Vote 9,Vote 10,Vote 11,Vote 12,Vote 13,Vote 14,Vote 15,Vote 16
2,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [0]:
# Function to evaluate the differences between the democrat and republican vote records.

def TwoSampleTtest(df1, df2):
    """Calculating the T-test statistic between 2 dataframes. Both dataframes must have
    the matching features in order for this function to work. NaN values are ommited.
    Return the result in form of the dataframe."""
    
    t_results = []
    
    for n in range(len(df1.columns)):
        calculation = ttest_ind(df1.iloc[:, n], df2.iloc[:, n], nan_policy='omit')
        t_results.append(calculation)
        
    t_results = pd.DataFrame(t_results)
    return t_results

In [7]:
# Actual Calculation.
# T-statistic above 0 means that Democrats voted more in favor of that particular issue.
# T-statistic below 0 means that Republicans voted more in favore of that particular issue.
# Adding the column with Vote# to make it easier to read. 

results = TwoSampleTtest(dem, rep)
results['Issue'] = column_names[1:]
results


,statistic,pvalue,Issue
0,9.205264,1.613440e-18,Vote 1
1,-0.088965,9.291557e-01,Vote 2
2,23.212777,2.070340e-77,Vote 3
3,-49.367082,1.994262e-177,Vote 4
4,-21.136693,5.600520e-68,Vote 5
5,-9.737576,2.393672e-20,Vote 6
6,12.526188,8.521033e-31,Vote 7
7,18.052093,2.824718e-54,Vote 8
8,16.437503,5.030793e-47,Vote 9
9,-1.735912,8.330248e-02,Vote 10


In [8]:
# Question - find an issue that democrats support more than republicans with p < 0.01
# Answer - Votes 1, 3, 7, 8, 9, 11, 15, 16

results.loc[(results['statistic'] > 0) & (results['pvalue'] < 0.01)]
    

,statistic,pvalue,Issue
0,9.205264,1.613440e-18,Vote 1
2,23.212777,2.070340e-77,Vote 3
6,12.526188,8.521033e-31,Vote 7
7,18.052093,2.824718e-54,Vote 8
8,16.437503,5.030793e-47,Vote 9
10,8.293604,1.575932e-15,Vote 11
14,12.853146,5.997697e-32,Vote 15
15,6.849455,3.652674e-11,Vote 16


In [9]:
# Question - find an issue that republicans support more than democrats with p < 0.01
# Answer - Votes 4, 5, 6, 12, 13, 14

results.loc[(results['statistic'] < 0) & (results['pvalue'] < 0.01)]

,statistic,pvalue,Issue
3,-49.367082,1.994262e-177,Vote 4
4,-21.136693,5.600520e-68,Vote 5
5,-9.737576,2.393672e-20,Vote 6
11,-20.500686,1.883420e-64,Vote 12
12,-13.510643,1.227858e-34,Vote 13
13,-16.342086,9.952343e-47,Vote 14


In [10]:
# Question - find an issue where the difference between republicans and democrats has p > 0.1 
# Answer - Vote 2

# NOTE - Vote 10 leans toward Republicans and has a P-value 0.08, so it doesn't neatly falls
# into any of the above categories, but that result is enough to fail to reject the Null Hypothesis that voting record between
# parties on that particular issue is different. 

results.loc[results['pvalue'] > 0.1]

,statistic,pvalue,Issue
1,-0.088965,0.929156,Vote 2
